In [1]:
import os
import selenium
from selenium import webdriver
import time
from PIL import Image
import re
import urllib.request

import io
import requests

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import ElementClickInterceptedException
from selenium import webdriver 

from pathlib import Path

%config Completer.use_jedi = False

# Manual

Define the URL of the query you're interested in and the name of the location

In [2]:
location_name = "algarve"

# create base folder
Path(f"data/{location_name}/").mkdir(parents=True, exist_ok=True)

In [3]:
search_url = \
    "https://www.jamesvillas.ie/destinations/portugal/algarve/vale-do-lobo/all-villas-to-rent-in-vale-do-lobo"

# Automated

Chrome options: seting `headless=True` will prevent chrome opening and will instead work in the background 

In [4]:
opts = webdriver.ChromeOptions()
opts.headless = False

In [5]:
# ignore the red, just printing logs of the driver downloading
driver = webdriver.Chrome(ChromeDriverManager().install(), options=opts)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Driver [/Users/gavintynan/.wdm/drivers/chromedriver/mac64/94.0.4606.61/chromedriver] found in cache


## Search

In [6]:
driver.get(search_url)

In [7]:
# there will be an element in the list for each destination within the url search
search_results = driver.find_elements_by_xpath("//*[starts-with(@id, 'result')]")

In [8]:
for i, result in enumerate(search_results):
    
    # one of the elements is a the overall container ignore it 
    if result.get_attribute("id") == 'results':
        continue
        
    # open a new tab by clicking the destination    
    result.click()

    # switch our drivers contenxt to said new tab
    driver.switch_to.window(driver.window_handles[i])
    
    # wait til next button appears on page
    next_button = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, 'owl-next'))
    )
    
    # way to parse pages html
    soup = BeautifulSoup(driver.page_source, 'html')
    
    # find center image div
    first_image = soup.find("div", {"class": "owl-item active center"})
    current_image = first_image
    
    while True: 
        
        # our images are actually the div surrounding it so have to get img from div
        for div in current_image.children:
            # extract url
            img = re.search(
                "(https?:\/\/.*\.(?:png|jpg))", div['style']).group(0)
            # use index for naming
            index = div['data-index']
            
            # download image
            urllib.request.urlretrieve(img, 
                                       "./data/{}/destination_{}_pic_{}.jpg".format(location_name, i, index))
        
        # go to next image on page
        next_button.click()
        
        soup = BeautifulSoup(driver.page_source, 'html')
        current_image = soup.find("div", {"class": "owl-item active center"})
        
        # once we cycle back to the first image stop for loop
        if first_image == current_image:
            break
                
    driver.switch_to.window(driver.window_handles[0])